# Initialize NMRlipids databank

In [2]:
import os
import sys
import numpy as np
import json
import matplotlib.pyplot as plt
import MDAnalysis
import urllib.request
import yaml

sys.path.insert(1, '../../Databank/Scripts/BuildDatabank/')
from databankLibrary import download_link, lipids_dict, databank

path = '../../Databank/Data/Simulations/'
db_data = databank(path)
systems = db_data.get_systems()

In [21]:
for system in systems:
    
    try:
        NPOPG = np.sum(system['COMPOSITION']['POPG']['COUNT'])
    except:
        NPOPG = 0    
    
    try:
        NPOPC = np.sum(system['COMPOSITION']['POPC']['COUNT'])
    except:
        NPOPC = 0
    
    if NPOPG == 0 and NPOPC == 0:
        continue
    
    print(NPOPG / (NPOPG + NPOPC))
    print(system['path'])

0.0
../../Databank/Data/Simulations/ef4/41c/ef441c853176fefb8f790026dc4473d80fbf8e61/3bd4758d5cad57a8bcbdaf35265550dd7fc0c01a/
0.0
../../Databank/Data/Simulations/702/d04/702d047ee14fb2c6b8ceb1666df45f0783272d8c/50161179ab704faeaf1644957545a77fe97a1063/
0.0
../../Databank/Data/Simulations/722/744/7227444b737b8023a50556d24a364903f2fee400/3623de48284f036bbb63c4806806493d755ce26a/
0.0
../../Databank/Data/Simulations/5a4/277/5a427728e5bcc18a6f162a8d325df4a9e4115250/f0fe9b9284aa683f9a419d9173d320cfa31d34cb/
0.0
../../Databank/Data/Simulations/543/ab3/543ab3c1266ace288f9b39ed561851dfcbf5746d/fa7f266129d60fe835af632f66de4d1cd594b5e5/
0.0
../../Databank/Data/Simulations/54b/650/54b6503812c7cd81137b04d302569bcd08aca03b/b56c1f05e72a73558cf1a7ac399a8420648399be/
0.0
../../Databank/Data/Simulations/e85/9ac/e859acb19b28003d353bf285516b6c978a8043cc/5b37230ffd595558eb9e9b2e5e82a47a223a34c2/
0.0
../../Databank/Data/Simulations/e20/8a2/e208a2f0435f9474b59a922deae73aff8d646c28/662f12d507ed9d155ec61736f5